In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import os

# ==========================================
# 1. PERSIAPAN DATA & DEFINISI MAKNA
# ==========================================

# Nama file dataset baru Anda
file_path = 'dataset_uji_sentimen_variasi_fix.csv'

if not os.path.exists(file_path):
    print(f"Error: File '{file_path}' tidak ditemukan.")
    raise FileNotFoundError(f"Pastikan file {file_path} sudah ada di direktori yang sama.")

df = pd.read_csv(file_path)

# Definisi Ground Truth berdasarkan pola kalimat di dataset baru Anda
# Kita memetakan contoh-contoh kalimat unik ke label sentimen yang relevan
mapping_makna = {
    'Layanan ini saya gunakan saat sedang sibuk': 'Netral',
    'Saya mengakses aplikasi pada waktu tertentu': 'Netral',
    'Saya membuka aplikasi hanya untuk melihat informasi': 'Netral',
    'Produk digunakan sesuai dengan fungsinya': 'Positif',
    'Produk ini saya beli karena rekomendasi teman': 'Positif',
    'Saya menggunakan layanan ini secara rutin': 'Positif',
    'Saya memanfaatkan fitur yang tersedia di aplikasi': 'Positif',
    'Saya hanya mencoba layanan ini sebentar': 'Netral',
    'Aplikasi ini dipakai untuk keperluan sehari-hari': 'Positif',
    'Saya menggunakan fitur utama yang tersedia': 'Positif',
    'Layanan ini digunakan tanpa kendala berarti': 'Positif',
    'Saya mencoba aplikasi ini untuk kebutuhan pribadi': 'Netral',
    'Saya mengakses layanan ini melalui ponsel': 'Netral',
    'Saya membuka aplikasi untuk mengecek status': 'Netral',
    'Saya melakukan transaksi melalui aplikasi tersebut': 'Positif',
    'Aplikasi ini digunakan untuk tujuan tertentu': 'Netral',
    'Aplikasi ini saya gunakan untuk pertama kali': 'Netral',
    'Produk ini saya gunakan sesuai kebutuhan': 'Positif',
    # Tambahan jika ada variasi lain yang muncul
    'Aplikasi sering mengalami gangguan': 'Negatif',
    'Kinerja aplikasi terasa lambat': 'Negatif',
    'Saya tidak puas menggunakan aplikasi ini': 'Negatif',
    'Tampilan aplikasi membingungkan': 'Negatif'
}

# Memberikan label awal untuk proses belajar model
# Jika kalimat tidak ada di mapping, kita beri label 'Netral' sebagai default
df['label'] = df['kalimat'].map(mapping_makna).fillna('Netral')

# ==========================================
# 2. PREPROCESSING & TOKENISASI
# ==========================================

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

df['cleaned_text'] = df['kalimat'].apply(clean_text)

# Membangun Kosakata (Vocabulary)
all_words = ' '.join(df['cleaned_text']).split()
vocab = sorted(list(set(all_words)))
word_to_idx = {word: i + 1 for i, word in enumerate(vocab)} # 0 untuk padding
idx_to_word = {i: word for word, i in word_to_idx.items()}

def tokenize(text, max_len=20):
    tokens = [word_to_idx[w] for w in text.split() if w in word_to_idx]
    if len(tokens) < max_len:
        tokens = tokens + [0] * (max_len - len(tokens))
    else:
        tokens = tokens[:max_len]
    return tokens

# Konversi ke format numerik
X_data = np.array(df['cleaned_text'].apply(lambda x: tokenize(x)).tolist())
le = LabelEncoder()
y_data = le.fit_transform(df['label'])

# ==========================================
# 3. ARSITEKTUR MODEL TRANSFORMER
# ==========================================

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, num_classes, max_len):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_len, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        # x shape: (batch_size, max_len)
        x = self.embedding(x) + self.pos_embedding
        x = self.transformer_encoder(x)
        x = x.mean(dim=1) # Global Average Pooling
        logits = self.fc(x)
        return logits

# Hyperparameters
VOCAB_SIZE = len(vocab) + 1
EMBED_DIM = 64
NUM_HEADS = 8
NUM_LAYERS = 2
NUM_CLASSES = len(le.classes_)
MAX_LEN = 20

# ==========================================
# 4. PELATIHAN MODEL
# ==========================================

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)

class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.LongTensor(X)
        self.y = torch.LongTensor(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(TextDataset(X_train, y_train), batch_size=16, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TransformerClassifier(VOCAB_SIZE, EMBED_DIM, NUM_HEADS, NUM_LAYERS, NUM_CLASSES, MAX_LEN).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
print(f"Memulai pelatihan model pada {device}...")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# ==========================================
# 5. EVALUASI & OUTPUT HASIL
# ==========================================

model.eval()
with torch.no_grad():
    all_input_tensor = torch.LongTensor(X_data).to(device)
    raw_outputs = model(all_input_tensor)
    _, predicted_indices = torch.max(raw_outputs, 1)
    df['hasil_label_transformer'] = le.inverse_transform(predicted_indices.cpu().numpy())

output_file = 'hasil_uji_sentimen_variasi_transformer.csv'
df[['kalimat', 'hasil_label_transformer']].to_csv(output_file, index=False)

print("\n--- PROSES SELESAI ---")
print(f"Data diproses: {len(df)} baris")
print(f"Hasil disimpan di: {output_file}")
print("\nSampel Hasil Prediksi (15 baris pertama):")
print(df[['kalimat', 'hasil_label_transformer']].head(15))

Memulai pelatihan model pada cuda...
Epoch [10/50], Loss: 0.0008
Epoch [20/50], Loss: 0.0003
Epoch [30/50], Loss: 0.0002
Epoch [40/50], Loss: 0.0001
Epoch [50/50], Loss: 0.0001

--- PROSES SELESAI ---
Data diproses: 400 baris
Hasil disimpan di: hasil_uji_sentimen_variasi_transformer.csv

Sampel Hasil Prediksi (15 baris pertama):
                                              kalimat hasil_label_transformer
0          Layanan ini saya gunakan saat sedang sibuk                  Netral
1         Saya mengakses aplikasi pada waktu tertentu                  Netral
2   Saya membuka aplikasi hanya untuk melihat info...                  Netral
3            Produk digunakan sesuai dengan fungsinya                 Positif
4         Saya mengakses aplikasi pada waktu tertentu                  Netral
5       Produk ini saya beli karena rekomendasi teman                 Positif
6          Layanan ini saya gunakan saat sedang sibuk                  Netral
7           Saya menggunakan layanan ini seca